<a href="https://colab.research.google.com/github/pawarharshit/tensorflow_programs/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [2]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.imdb.load_data()

17465344/17464789 [==============================] - 0s 0us/step


In [3]:
len(X_train)

25000

In [4]:
word_index=tf.keras.datasets.imdb.get_word_index()

1646592/1641221 [==============================] - 0s 0us/step


In [0]:
id_to_word = {id_ + 3: word for word,id_ in word_index.items()}

In [0]:
for id_,token in enumerate(("<pad>","<sos>","<unk>")):
    id_to_word[id_] = token

In [7]:
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [8]:
import tensorflow_datasets as tfds

dataset,info = tfds.load("imdb_reviews",as_supervised=True,with_info=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete46BPCU/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete46BPCU/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete46BPCU/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_size = info.splits['train'].num_examples

In [0]:
def preprocess(X_batch,y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"),y_batch 

In [0]:
from collections import Counter
vocabulary = Counter()

for X_batch,y_batch in dataset['train'].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [20]:
vocabulary.most_common(3)

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [0]:
vocab_size = 10000
truncated_vocabulary = [word for word,count in vocabulary.most_common(vocab_size)]

In [0]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary),dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words,word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init,num_oov_buckets)

In [26]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

In [0]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch   

In [0]:
train_set = dataset['train'].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [0]:
embed_size = 128
model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size+num_oov_buckets,embed_size,input_shape=[None]),
            tf.keras.layers.GRU(128,return_sequences=True),
            tf.keras.layers.GRU(128),
            tf.keras.layers.Dense(1,activation='sigmoid')
])

In [31]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])
history = model.fit(train_set,epochs=5)

Epoch 1/5
782/782 [==============================] - 113s 145ms/step - loss: 0.6173 - accuracy: 0.6318
Epoch 2/5
782/782 [==============================] - 109s 139ms/step - loss: 0.4374 - accuracy: 0.8026
Epoch 3/5
782/782 [==============================] - 109s 140ms/step - loss: 0.3314 - accuracy: 0.8646
Epoch 4/5
782/782 [==============================] - 110s 140ms/step - loss: 0.2372 - accuracy: 0.9117
Epoch 5/5
782/782 [==============================] - 109s 140ms/step - loss: 0.1735 - accuracy: 0.9412
